In [ ]:
import re
import pickle 
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset , DataLoader

In [ ]:
#read pickle file
pickle_file = open("/content/sample_data/plots_text.pickle","rb")
movie_file = pickle.load(pickle_file)

# New Section

In [ ]:
movie_file[0]

'barry is a private with the 101st airborne division of the united states army, stationed at fort campbell, kentucky. calpernia works as a showgirl at a transgender revue in nashville, tennessee when the two met in 1999. barry\'s roommate justin fisher  brings barry to the club where she performs. when barry and calpernia begin seeing each other regularly, fisher begins spreading rumors on base about their relationship, which appeared to be a violation of the military\'s "don\'t ask, don\'t tell" policy about discussing the sexual orientation of military personnel. barry faces increasing harassment and pressure, which explode into violence over fourth of july weekend. while calpernia performs in a pageant in nashville, barry is beaten to death in his sleep with a baseball bat by calvin glover, who had been goaded by fisher into committing the crime. the film ends with a discussion of the aftermath.'

In [ ]:
len(movie_file)

500

In [ ]:
#clean data
clean_movie = [re.sub("[^a-z 0-9 ]","",i) for i in movie_file]


In [ ]:
clean_movie[0]

'barry is a private with the 101st airborne division of the united states army stationed at fort campbell kentucky calpernia works as a showgirl at a transgender revue in nashville tennessee when the two met in 1999 barrys roommate justin fisher  brings barry to the club where she performs when barry and calpernia begin seeing each other regularly fisher begins spreading rumors on base about their relationship which appeared to be a violation of the militarys dont ask dont tell policy about discussing the sexual orientation of military personnel barry faces increasing harassment and pressure which explode into violence over fourth of july weekend while calpernia performs in a pageant in nashville barry is beaten to death in his sleep with a baseball bat by calvin glover who had been goaded by fisher into committing the crime the film ends with a discussion of the aftermath'

In [ ]:
#create sequence 
def create_seq(text,seq_len=8):

  sequence = []

  if len(text.split()) > seq_len:
    for i in range(seq_len,len(text.split())): #(8,50)
      #select the sequnece of tokens
      seq = text.split()[i-seq_len:i+1] #(0-9,1-10,2-11,3-12,4-13)
      sequence.append(" ".join(seq))

    return sequence


  else:
    return [text]

In [ ]:
#["I","Love","my","country"]
#["I","Love"]
#["Love","my"]
#["my","country"]

#[I LOVE MY COUNTRY]

In [ ]:
seqs = [create_seq(i) for i in clean_movie]

In [ ]:
#mergs list-of-list into a single list
seqs_s = sum(seqs,[])

In [ ]:
len(seqs)

500

In [ ]:
seqs_n = []

for i in seqs:
  for j in i:
    seqs_n.append(j)

In [ ]:
seq_nc = [j for i in seqs for j in i]

In [ ]:
len(seqs_n)

151501

In [ ]:
len(seq_nc)

151501

In [ ]:
len(seqs_s)

151501

In [ ]:
#creat input & target sequence
x = []
y = []

for i in seqs_s:
  x.append(" ".join(i.split()[:-1]))
  y.append(" ".join(i.split()[1:]))

In [ ]:
len(x) , len(y)

(151501, 151501)

In [ ]:
# create int-to-word

int2word = {}
c = 0
uni_word = set(" ".join(clean_movie).split())

# for i in uni_word:
#   int2word[c] = i
#   c += 1

for index,word in enumerate(uni_word):
  int2word[index] = word


int_2_word = {index:word for index,word in enumerate(uni_word)}

In [ ]:
word2int = {word:index for index,word in int2word.items()}

In [ ]:
len(int_2_word)

16531

In [ ]:
len(int2word)

16531

In [ ]:
len(word2int)

16531

In [ ]:
vocab_size = len(int2word)
vocab_size

16531

In [ ]:
def get_int_seq(seq):
  return [word2int[w] for w in seq.split()]

In [ ]:
#x and y covert in into int
x_int = [get_int_seq(i) for i in x]
y_int = [get_int_seq(j) for j in y]

In [ ]:
#convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [ ]:
x_int[0]

array([ 8529, 10318, 11945, 13967,  9685, 12862,   231,  3276])

In [ ]:
# def seq_int(seq):
#   return [word2int[word] for sen in seq for word in sen.split()]
#   #[[word2int(word)for sen in seq for word in sen.split()]]


# x_int_1 = seq_int(x)
# #y_int_2 = seq_int(y)

In [ ]:
# b = []
# for sen in x:
#   a = []
#   for j in sen.split():
#     a.append(word2int[j])

#   b.append(a)

In [ ]:
len(x_int) , len(y_int)

(151501, 151501)

In [ ]:
#inputs,targets = torch.from_numpy(x_int) , torch.from_numpy(y_int)

In [ ]:
x_int.dtype

dtype('int64')

In [ ]:
inputs = x_int
targets = y_int

In [ ]:
#train , test & validation
train_ratio = 0.8
valid_ratio = (1-train_ratio)/2
#test_ratio = (1 -(train_ratio+valid_ratio))

total = len(x_int)
train_cutoff = int((total*train_ratio))
valid_cutoff = int((total*(1-valid_ratio))) #(50*(0.9)) 45


#numpy to tensor
train_x, train_y = torch.Tensor(inputs[:train_cutoff]).long() , torch.Tensor(targets[:train_cutoff]).long()
valid_x , valid_y = torch.Tensor(inputs[train_cutoff:valid_cutoff]).long() , torch.Tensor(targets[train_cutoff:valid_cutoff]).long()
test_x , test_y = torch.Tensor(inputs[valid_cutoff:]).long() , torch.Tensor(targets[valid_cutoff:]).long()


train_data = TensorDataset(train_x,train_y)
valid_data = TensorDataset(valid_x ,valid_y)
test_data = TensorDataset(test_x,test_y)

batch_size = 32


train_loader = DataLoader(train_data,batch_size = batch_size,shuffle=True,drop_last=True)
valid_loader = DataLoader(valid_data,batch_size = batch_size,shuffle=True,drop_last=True)
test_loader = DataLoader(test_data,batch_size = batch_size,shuffle=True,drop_last=True)

In [ ]:
# n = 130
# b = 8
# 130/8 (2)

In [ ]:
for x , y in train_loader:
  print(x.shape,y.shape)
  break

torch.Size([32, 8]) torch.Size([32, 8])


In [ ]:
# def get_batches(x_arr,y_arr,batch_size):
#   c = 0
#   for i in range(batch_size,x_arr.shape[0],batch_size):
#     x = x_arr[c:i,:] #[0:32,:] , [32:64,:] , [64:96,:]
#     y = y_arr[c:i,:] #[0:32,:] , [32:64,:] , [64:96,:]

#     c = i #[32]

#     yield x,y

In [ ]:
x_int.shape

(151501, 8)

In [ ]:
class TextGenerationLSTM(nn.Module):

  def __init__(self,n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25):
    super().__init__()
    self.n_vocab = n_vocab
    self.n_embed = n_embed
    self.n_hidden = n_hidden 
    self.drop_pr = drop_pr 
    self.n_layers = n_layers

    self.embedding = nn.Embedding(n_vocab,n_embed)
    self.lstm = nn.LSTM(n_embed,n_hidden,n_layers,batch_first = True)
    self.dropout = nn.Dropout(drop_pr)
    self.fc = nn.Linear(n_hidden,n_vocab)

  #(input , (hidden_state & cell state))
  def forward(self,x,hidden):
    
    #pass the input through embedding layers
    embedded = self.embedding(x)

    #Get the output and new hidden state and cell state
    n_out , hidden = self.lstm(embedded,hidden) #(lstm) (input,(hidden_state,cell_state)) 

    #pass the droupout
    out = self.dropout(n_out)

    #reshape
    #out = out.contingous().view(-1,self.n_hidden)
    out = out.reshape(-1,self.n_hidden)

    out = self.fc(out)

    return out,hidden


  def init_hidden(self,batch_size):

    '''initialize the hidden and cell state with size (n_layers * batch_size * n_hidden) ,
    initialize to zero for both hidden and cell state of LSTM model '''

    #hidden layer = Total no of neurons in hidden state (hidden_state value)

    # LSTM (hidden state and cell state) (hidden state , cell (memory))
    #120 neurons state ()

    weights = next(self.parameters()).data

    if (torch.cuda.is_available()):
      hidden = (weights.new(self.n_layers , batch_size,self.n_hidden).zero_().cuda(),
         weights.new(self.n_layers,batch_size,self.n_hidden).zero_().cuda())

    else:
      hidden = (weights.new(self.n_layers , batch_size,self.n_hidden).zero_(),
         weights.new(self.n_layers,batch_size,self.n_hidden).zero_())
      
    return hidden

In [ ]:
#n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25
n_vocab = len(int2word)
n_embed = 200
n_layers = 3
n_hidden = 128

net = TextGenerationLSTM(n_vocab,n_embed,n_hidden,n_layers,drop_pr = 0.25)

In [ ]:
net
net.cuda()

TextGenerationLSTM(
  (embedding): Embedding(16531, 200)
  (lstm): LSTM(200, 128, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=16531, bias=True)
)

In [ ]:
print(net)

TextGenerationLSTM(
  (embedding): Embedding(16531, 200)
  (lstm): LSTM(200, 128, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=16531, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def acc(pred,label):
  pred = torch.round(pred)
  return torch.sum(pred==label).item()

In [ ]:
'''The error may caused by your training data not being divisible by batch size. 
Suppose your training data has 130 items and batch size is 8, the last batch will have only 2 
(remainder of 130/8) items. Therefore by setting drop_last to True, the 2 items will be ignored.'''

'The error may caused by your training data not being divisible by batch size. \nSuppose your training data has 130 items and batch size is 8, the last batch will have only 2 \n(remainder of 130/8) items. Therefore by setting drop_last to True, the 2 items will be ignored.'

In [ ]:
def train_sentiment(net,loss_fn,optimizer,epochs=20):

  for epoch in range(epochs):
    h = net.init_hidden(batch_size=32)
    #print(h[0].shape)
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0


    net.train()

    for input,labels in train_loader:
      h = tuple([i.data for i in h]) #tuple([hidden_state,cell_state])
      input , labels = input.cuda() , labels.cuda()
      #print(h[0].shape)
      # print(input.shape,labels.shape)
     
    
      output,h = net(input,h)
      optimizer.zero_grad()

      loss = loss_fn(output,labels.view(-1))
      loss.backward()
      train_loss += loss.item()

      # print(labels.shape)
      #print(output.shape)
      #accuracy = acc(output,labels.view(-1))
      #train_acc += accuracy

      optimizer.step()

    net.eval()

    for input,labels in valid_loader:
      input , labels = input.cuda() , labels.cuda()
      h = tuple([i.data for i in h]) #tuple([hidden_state,cell_state])
      output,h = net(input,h)
      

      loss = loss_fn(output,labels.view(-1))
      valid_loss += loss.item()

      # accuracy = acc(output,labels)

      # valid_acc += accuracy

    print("Epoch {} , Train_loss : {:.2f},valid_loss : {:.2f}".format(
        epoch+1 , train_loss/len(test_loader),valid_loss/len(valid_loader)))

In [ ]:
train_sentiment(net,loss_fn,optimizer,epochs=40)

Epoch 1 , Train_loss : 51.95,valid_loss : 7.54
Epoch 2 , Train_loss : 44.57,valid_loss : 8.18
Epoch 3 , Train_loss : 40.34,valid_loss : 8.81
Epoch 4 , Train_loss : 37.31,valid_loss : 9.46
Epoch 5 , Train_loss : 35.07,valid_loss : 10.04
Epoch 6 , Train_loss : 33.33,valid_loss : 10.49
Epoch 7 , Train_loss : 31.94,valid_loss : 10.82
Epoch 8 , Train_loss : 30.77,valid_loss : 11.18
Epoch 9 , Train_loss : 29.82,valid_loss : 11.42
Epoch 10 , Train_loss : 28.95,valid_loss : 11.65
Epoch 11 , Train_loss : 28.21,valid_loss : 11.89
Epoch 12 , Train_loss : 27.52,valid_loss : 12.12
Epoch 13 , Train_loss : 26.93,valid_loss : 12.34
Epoch 14 , Train_loss : 26.39,valid_loss : 12.56
Epoch 15 , Train_loss : 25.87,valid_loss : 12.69
Epoch 16 , Train_loss : 25.41,valid_loss : 12.85
Epoch 17 , Train_loss : 25.01,valid_loss : 13.02
Epoch 18 , Train_loss : 24.59,valid_loss : 13.15
Epoch 19 , Train_loss : 24.23,valid_loss : 13.31
Epoch 20 , Train_loss : 23.92,valid_loss : 13.37
Epoch 21 , Train_loss : 23.58,val

In [ ]:
# def train(net,epochs=10,batch_size=32,clip=1):

#   net.train()
#   c = 0
  

#   for epoch in range(epochs):

#     #initialize hidden state/cell state
#     h = net.init_hidden(batch_size)

#     # c = 0 #h - (hidden ,cell state ) (d)

#     # for i in h:
#     #   #print(i)
#     #   c += 1

#     # print(c)
    
#     for x,y in get_batches(x_int,y_int,batch_size):
#       c += 1

#       #(32,8)

#       #convert numoy to tensor
#       inputs,targets = torch.from_numpy(x) , torch.from_numpy(y)


#       #Tensor to cuda
#       inputs , targets = inputs.cuda() , targets.cuda()

#       #detach hidden states
#       h = tuple([i.data for i in h]) #tuple([hidden_state,cell_state])

#       output , h = net(inputs,h) #(hidden,cellstate)

#       # gradient = 10 ( Wn = Wo - lr*gr) (4,10,1-000,10000,1000) (-1,1) max(-x,1=x) max(0,x)

#       loss = loss_fn(output,targets.view(-1))

#       loss.backward() #claculating gradeint 

#       nn.utils.clip_grad_norm_(net.parameters(),clip)

#       opt.step()


#     if c%32==0:
#         print("Epoch {} / {} , loss {:.2f}".format(epoch ,epochs,loss))


#       #a = targets.view(-1)
#       #print(a)
#       #print(a.shape)

#       #print(loss)
#       #print(output[1].shape)
#       #print(output.shape)
#       # print(targets.shape)
#       # print(inputs.shape)

#     #[0,0,2,0.3,0.4,50

In [ ]:
def predict(net,tkn,h = None):

  # tensor inputs

  # net.cuda()

  # net.eval()

  # h = net.init_hidden(1)

  x = np.array([[word2int[tkn]]])
  inputs = torch.from_numpy(x)

  # To gpu
  inputs = inputs.cuda()

  # detech hidden & cell state 
  h = tuple([i.data for i in h])

  # get the output of the model
  out , h = net(inputs,h)




  # get the token proba
  p = F.softmax(out,dim=1).data # cuda

  p = p.cpu()
  p = p.numpy()

  p = p.reshape(p.shape[1],)

  #get indices of next or top 3 values
  top_n_indx = p.argsort()[-1:][::-1][0]

  #random selct one of the two indecies 

  return int2word[top_n_indx] , h

In [ ]:
#function generate text
def sample(net,size,prime="it is good"):

  # To GPU
  net.cuda()

  net.eval()

  #batch size as 1
  h = net.init_hidden(1)

  tokens = prime.split() # ["it","is","good"]

  #predict next token
  for t in tokens:
    token , h = predict(net,t.lower(),h)

  tokens.append(token)

  #predict subseuit toekns

  for i in range(size):
    token , h  = predict(net,tokens[-1],h)
    tokens.append(token)

  return " ".join(tokens)

In [ ]:
sample(net,100,"we have to find the")

'we have to find the capsule at the beginning of the film dille becomes a professor at the hands of the police station but the man who has been rebuilding her career in love with the painters wife and rapes her he discovers that she is a good student and gets admitted to an argument that caused the car and the film ends with a quote from diggler all the two men pair with force 10 and forces her to be the object of the captains secret affection nothing in the park while the assassin stands emotionless in front of the group katie stabs him in'